In [1]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline

import time
import pickle
from imp import reload
from os.path import join

import pandas as pd
import seaborn as sns
import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

from data_loader import DataLoader
import medim
reload(medim);

In [2]:
raw_path = '/home/mount/neuro-t01-hdd/Brats2017/data/raw/'

data_loader = DataLoader(raw_path)
patients = data_loader.patients

In [3]:
processed_path = '/mount/export/Brats2017/data/processed'
mscans = []
segmentations = []

for patient in tqdm(patients[:5]):
    filename = join(processed_path, patient)
    
    mscans.append(np.load(filename+'_mscan.npy'))
    segmentations.append(np.load(filename+'_segmentation.npy'))

100%|██████████| 5/5 [00:01<00:00,  4.29it/s]


In [12]:
n_classes = 4

In [4]:
patch_size_x = np.array([10, 10, 10])
patch_size_y = np.array([6, 6, 6])


def uniform(mscans, segms, *, batch_size, patch_size_x, patch_size_y):
    """Patch iterator with uniformed distribution over spatial dimensions"""
    assert np.all(patch_size_x % 2 == patch_size_y % 2)
    patch_size_pad = (patch_size_x - patch_size_y) // 2

    n = len(mscans)
    n_mods = mscans[0].shape[0]

    max_spatial_idx = np.array(
        [list(s.shape[1:]) for s in mscans]) - patch_size_x + 1

    x_batch = np.zeros((batch_size, n_mods, *patch_size_x), dtype=np.float32)
    y_batch = np.zeros((batch_size, *patch_size_y), dtype=np.int64)

    while True:
        idx = np.random.randint(n, size=batch_size)
        start_idx = np.random.rand(batch_size, 3) * max_spatial_idx[idx]
        start_idx = np.int32(np.floor(start_idx))
        for i in range(batch_size):
            s = start_idx[i]
            slices = [...] + [slice(s[k], s[k] + patch_size_x[k]) for k in
                              range(3)]
            x_batch[i] = mscans[idx[i]][slices]

            s = start_idx[i] + patch_size_pad
            slices = [slice(s[k], s[k] + patch_size_y[k]) for k in range(3)]
            y_batch[i] = segms[idx[i]][slices]
        yield np.array(x_batch), np.array(y_batch)
        
        
def min_padding(mscan, padding):
    padding = np.array([0] + list(padding))
    padding = np.repeat(padding[:, None], 2, axis=1)
    
    return np.pad(mscan, padding, mode='minimum')

In [463]:
patch_size_x = np.array([10, 10, 10])
patch_size_y = np.array([6, 6, 6])

def uniform(mscans, msegms, *, batch_size, patch_size_x, patch_size_y):
    """Patch iterator with uniformed distribution over spatial dimensions"""
    assert np.all(patch_size_x % 2 == patch_size_y % 2)
    patch_size_pad = (patch_size_x - patch_size_y) // 2

    n = len(mscans)

    max_spatial_idy = np.array(
        [list(s.shape[1:]) for s in mscans]) - patch_size_y + 1

    x_batch = np.zeros((batch_size, mscans[0].shape[0], *patch_size_x), dtype=np.float32)
    y_batch = np.zeros((batch_size, msegms[0].shape[0], *patch_size_y), dtype=np.float32)

    while True:
        idy = np.random.randint(n, size=batch_size)
        start_idy = np.random.rand(batch_size, 3) * max_spatial_idy[idy]
        start_idy = np.int32(np.floor(start_idy))
        for i in range(batch_size):
            s = start_idy[i]
            slices = [...] + [slice(s[k], s[k] + patch_size_y[k])
                              for k in range(3)]
            y_batch[i] = msegms[idy[i]][slices]

            s = start_idy[i] - patch_size_pad
            e = s + patch_size_x
            padding_l = [0] + [-s[i] if s[i] < 0 else 0 for i in range(s.shape[0])]
            padding_r = [0] + [-mscans[idy[i]].shape[k+1] + e[k] if e[k] >= mscans[idy[i]].shape[k+1] else 0 \
                               for k in range(e.shape[0])]
            padding = list(zip(padding_l, padding_r))
            slices = [...] + [slice(max(s[k], 0), min(e[k], mscans[idy[i]].shape[k+1])) 
                              for k in range(3)]
            
            min_const = np.amin(mscans[idy[i]])
            x_batch[i] = np.pad(mscans[idy[i]][slices], padding, mode='constant', constant_values=min_const)
        yield np.array(x_batch), np.array(y_batch)

In [493]:
patch_size_x_ext = np.array([20, 20, 20])
patch_size_x = np.array([10, 10, 10])
patch_size_y = np.array([6, 6, 6])

def uniform_ext(mscans, msegms, *, batch_size, patch_size_x_ext, patch_size_x, patch_size_y):
    """Patch iterator with uniformed distribution over spatial dimensions"""
    assert np.all(patch_size_x % 2 == patch_size_y % 2)
    patch_size_pad = (patch_size_x - patch_size_y) // 2
    patch_size_pad_ext = (patch_size_x_ext - patch_size_y) // 2

    n = len(mscans)

    max_spatial_idy = np.array(
        [list(s.shape[1:]) for s in mscans]) - patch_size_y + 1

    x_batch_ext = np.zeros((batch_size, mscans[0].shape[0], *patch_size_x_ext), dtype=np.float32)
    x_batch = np.zeros((batch_size, mscans[0].shape[0], *patch_size_x), dtype=np.float32)
    y_batch = np.zeros((batch_size, msegms[0].shape[0], *patch_size_y), dtype=np.int64)

    while True:
        idy = np.random.randint(n, size=batch_size)
        start_idy = np.random.rand(batch_size, 3) * max_spatial_idy[idy]
        start_idy = np.int32(np.floor(start_idy))
        for i in range(batch_size):
            s = start_idy[i]
            slices = [...] + [slice(s[k], s[k] + patch_size_y[k]) for k in
                              range(3)]
            y_batch[i] = msegms[idy[i]][slices]

            s = start_idy[i] - patch_size_pad
            e = s + patch_size_x
            padding_l = [0] + np.min( [-s[i] if s[i] < 0 else 0 for i in range(s.shape[0])]
            padding_r = [0] + [-mscans[idy[i]].shape[k+1] + e[k] if e[k] >= mscans[idy[i]].shape[k+1] else 0 for k in range(e.shape[0])]
            padding = list(zip(padding_l, padding_r))
            slices = [...] + [slice(max(s[k], 0), min(e[k], mscans[idy[i]].shape[k+1])) 
                              for k in range(3)]
            
            min_const = np.amin(mscans[idy[i]])
            x_batch[i] = np.pad(mscans[idy[i]][slices], padding, mode='constant', constant_values=min_const)
            
            s = start_idy[i] - patch_size_pad_ext
            e = s + patch_size_x_ext
            padding_l = [0] + [-s[i] if s[i] < 0 else 0 for i in range(s.shape[0])]
            padding_r = [0] + [-mscans[idy[i]].shape[k+1] + e[k] if e[k] >= mscans[idy[i]].shape[k+1] else 0 \
                               for k in range(e.shape[0])]
            padding = list(zip(padding_l, padding_r))
            slices = [...] + [slice(max(s[k], 0), min(e[k], mscans[idy[i]].shape[k+1])) 
                              for k in range(3)]
            
            min_const = np.amin(mscans[idy[i]])
            x_batch_ext[i] = np.pad(mscans[idy[i]][slices], padding, mode='constant', constant_values=min_const)
        yield np.array(x_batch_ext), np.array(x_batch), np.array(y_batch)

SyntaxError: invalid syntax (<ipython-input-493-6f71792e02dc>, line 33)

In [150]:
min([np.amin(mscans[i]) for i in range(len(mscans))])

-6.1083965

In [239]:
mscans[0].shape

(4, 140, 172, 145)

In [8]:
segmentations[0].shape

(3, 140, 172, 145)

In [34]:
a = min_padding(mscans[0], [100, 10, 10], 2)

[[  0   0]
 [100 100]
 [ 10  10]
 [ 10  10]]


In [10]:
a.shape

(4, 340, 192, 165)

In [77]:
next(iter)

(4, 142, 174, 147)

In [380]:
mscans[2].shape

(4, 142, 159, 138)

In [464]:
batch_size = 2
batch_iter = uniform(mscans, segmentations, batch_size=batch_size,
                     patch_size_x=patch_size_x, patch_size_y=patch_size_y)

In [484]:
#%%timeit

x_batch, y_batch = next(batch_iter)

In [489]:
batch_size = 2
batch_iter = uniform_ext(mscans, segmentations, batch_size=batch_size, patch_size_x_ext=patch_size_x_ext,
                     patch_size_x=patch_size_x, patch_size_y=patch_size_y)

In [490]:
#%%timeit

x_batch_ext, x_batch, y_batch = next(batch_iter)

In [491]:
print(x_batch_ext.shape, x_batch.shape, y_batch.shape)

(2, 4, 20, 20, 20) (2, 4, 10, 10, 10) (2, 3, 6, 6, 6)


In [485]:
x_batch.shape

(2, 4, 10, 10, 10)

In [486]:
y_batch.shape

(2, 3, 6, 6, 6)

In [102]:
def one_hot(x):
    enc = np.eye(n_classes)[x]
    return np.rollaxis(enc, 4, 1)

In [37]:
one_hot(segmentations[0]).shape

(4, 140, 172, 145)

In [19]:
x_batch[0].mean(axis=(1, 2, 3))

array([-0.78120344, -1.0260286 , -1.05939626, -0.63056652])